In [2]:
import pandas as pd

In [ ]:
data = pd.read_csv('trajectory_pairs.csv')

column_name_list = data.columns

# 判断是否有该类数据
assert "rejected" in column_name_list,  "Column 'rejected' not found in the CSV file."
assert "preferred" in column_name_list, "Column 'preferred' not found in the CSV file."

